In [1]:
import numpy as np

# Define the objective function f(x1, x2)
def objective(x):
    return x[0]**2 + x[1]**2 + 2*x[0] - 2*x[1] + 1

# Define the constraint function g(x1, x2)
def constraint(x):
    return x[0]**2 + x[1]**2 - 2  # g(x1, x2) = 0

# Compute the gradient of the objective function
def gradient_of_objective(x):
    df_dx1 = 2*x[0] + 2  # derivative w.r.t x1
    df_dx2 = 2*x[1] - 2  # derivative w.r.t x2
    return np.array([df_dx1, df_dx2])

# Compute the gradient of the constraint function
def gradient_of_constraint(x):
    dg_dx1 = 2*x[0]  # derivative w.r.t x1
    dg_dx2 = 2*x[1]  # derivative w.r.t x2
    return np.array([dg_dx1, dg_dx2])

# Augmented Lagrangian Method (ALM) without BFGS (gradient descent-based)
def augmented_lagrange_multiplier(objective, constraint, gradient_of_objective, gradient_of_constraint, x0, max_iter=100, rho=0.1, tol=1e-6, alpha=0.01, penalty_growth_rate=1.01):
    # Initial guess for x (x1, x2)
    x = np.array(x0)
    lambda_ = 0  # Initial guess for the Lagrange multiplier
    penalty = rho  # Penalty term

    # Iterate to update the solution and Lagrange multiplier
    for iteration in range(max_iter):
        # Compute gradients of the objective and constraint
        grad_f = gradient_of_objective(x)  # Gradient of the objective function
        grad_g = gradient_of_constraint(x)  # Gradient of the constraint function
        
        # Compute the gradient of the augmented Lagrangian
        grad_L = grad_f + lambda_ * grad_g + penalty * constraint(x) * grad_g  # Gradient of Augmented Lagrangian
        
        # Update the solution using gradient descent step
        x_new = x - alpha * grad_L
        
        # Ensure that the new solution doesn't lead to overflow or NaN values
        if np.any(np.isnan(x_new)) or np.any(np.isinf(x_new)):
            print("NaN or Inf encountered in solution update! Stopping optimization.")
            break

        # Update the Lagrange multiplier based on current constraint violation
        lambda_ += penalty * constraint(x_new)
        
        # Increase penalty term for stricter constraint satisfaction
        penalty = min(penalty * penalty_growth_rate, 1e6)  # Limit penalty growth
        
        # Check if x has sufficiently converged
        if np.linalg.norm(x_new - x) < tol and np.abs(constraint(x)) < tol:
            print(f"Converged after {iteration+1} iterations.")
            break
        
        # Update x for the next iteration
        x = x_new

    return x, objective(x), constraint(x)

# Example usage with an initial guess
x0 = [2.0, 2.0]  # Initial guess for x (x1, x2)
optimal_x, optimal_f, optimal_g = augmented_lagrange_multiplier(objective, constraint, gradient_of_objective, gradient_of_constraint, x0)

print("Optimal solution:", optimal_x)
print("Optimal objective value:", optimal_f)
print("Constraint value at optimal solution:", optimal_g)


Optimal solution: [-0.87411296  1.26144272]
Optimal objective value: -0.9158001575725232
Constraint value at optimal solution: 0.3553112148254658
